In [1]:
# warning filters
import warnings
warnings.filterwarnings("ignore", message="Pandas requires version")
warnings.filterwarnings("ignore", message="A NumPy version >=")

# general imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# anonymity library
import pycanon
from anjana.anonymity import k_anonymity, l_diversity, t_closeness

# ML models
from xgboost import XGBClassifier as XGB

# our generic functions
from utils import get_metrics, write_target_distribution_results_to_csv, get_generalization_levels, get_train_test_data

# our data-specific functions
from utils import clean_process_data, get_hierarchies

# our individual fairness metrics
from utils import lipschitz_fairness, similarity_fairness, neighborhood_consistency_fairness

# our config file
import config_experiments as cfg

# import folkatables
import folktables
from folktables import ACSDataSource

# ray for parallel processing (individual fairness is computationally expensive)
import ray 
import os
ray.init(num_cpus=os.cpu_count(), ignore_reinit_error=True)

# Get parameters from config file
lst_dataset = cfg.lst_dataset
lst_sensitive_attributes = cfg.lst_sensitive_attributes
max_seed = cfg.max_seed
test_size = cfg.test_size
fraction = cfg.fixed_fraction
supp_level = cfg.supp_level[1]
lst_deciles = cfg.lst_deciles
dic_methods_parameters = {
                         'k-anonymity': cfg.fixed_k,
                         'l-diversity': cfg.fixed_l,
                         't-closeness': cfg.fixed_t
                         } 

# Loop over several decile values
for decile in lst_deciles:
    print(f"decile for threshold target: {decile}")

    for dataset in ['adult']:#lst_dataset:

        # Main execution
        write_target_distribution_results_to_csv([], dataset=dataset, header=True)
        
        for protected_att in lst_sensitive_attributes[dataset]:

            # Loop over the three anonymization methods
            for method, anon_parameter in dic_methods_parameters.items():
                print(f"Method: {method}, Parameter: {anon_parameter}") 

                # read data
                if dataset == 'adult':

                    # Sensitive/target and protected attributes
                    sens_att = "income"
                    
                    # Read and process the data
                    data = pd.read_csv("adult_reconstruction.csv")
                    threshold_target = int(np.percentile(data[sens_att], decile))
                    data = clean_process_data(data, dataset, sens_att, protected_att, threshold_target)

                elif dataset == 'ACSIncome':

                    # Sensitive/target
                    sens_att = 'PINCP'

                    # Read and process the data
                    data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
                    acs_data = data_source.get_data()
                    Our_ACSIncome = folktables.BasicProblem(
                                                            features=[
                                                                    'AGEP',
                                                                    'COW',
                                                                    'SCHL',
                                                                    'MAR',
                                                                    'OCCP',
                                                                    'POBP',
                                                                    'RELP',
                                                                    'WKHP',
                                                                    'SEX',
                                                                    'RAC1P',
                                                                    ],
                                                            target='PINCP',
                                                            target_transform=lambda x: x,    
                                                            group=protected_att,
                                                            preprocess=folktables.adult_filter,
                                                            postprocess=lambda x: np.nan_to_num(x, -1),
                                                            )
                    features, target, _ = Our_ACSIncome.df_to_pandas(acs_data)
                    data = pd.concat([features, target.astype(int)], axis=1)
                    threshold_target = int(np.percentile(data[sens_att], decile))
                    full_data = clean_process_data(data, dataset, sens_att, protected_att, threshold_target)

                print("Dataset: {} with Protected Attribute: {}".format(dataset, protected_att))

                # Import/defining the hierarquies for each quasi-identifier. 
                hierarchies = get_hierarchies(data, dataset)

                # Define the quasi-identifiers (all columns except the sensitive attribute)
                quasi_ident = list(set(data.columns) - {sens_att})
                
                # Loop over several seeds            
                SEED = 0
                current_seed = 0
                while SEED < max_seed:
                    print(f"SEED: {SEED}, current_seed: {current_seed}")
                    data = full_data.sample(frac=fraction, random_state=current_seed) if dataset == 'ACSIncome' else data 

                    try:
                        # Split into train and test data
                        train_data, test_data = train_test_split(data, test_size=test_size, random_state=current_seed)

                        # Anonymize data
                        train_data_anon = k_anonymity(train_data, [], quasi_ident, cfg.fixed_k, supp_level, hierarchies)
                        if 'index' in train_data_anon.columns:
                            del train_data_anon['index'] 

                        # Assert that the level of k-anonymity is at least k
                        actual_k_anonymity = pycanon.anonymity.k_anonymity(train_data_anon, quasi_ident)
                        assert actual_k_anonymity >= cfg.fixed_k, f"k-anonymity constraint not met: Expected >= {cfg.fixed_k}, but got {actual_k_anonymity}"

                        if method == 'l-diversity':
                            # Apply l-diversity
                            train_data_anon = l_diversity(train_data_anon, [], quasi_ident, sens_att, cfg.fixed_k, anon_parameter, supp_level, hierarchies)

                            # Assert that the level of l-diversity is exactly met
                            actual_l_diversity = pycanon.anonymity.l_diversity(train_data_anon, quasi_ident, [sens_att])
                            assert actual_l_diversity == anon_parameter, f"l-diversity constraint not met: Expected == {anon_parameter}, but got {actual_l_diversity}"

                        if method == 't-closeness':
                            # Apply t-closeness
                            train_data_anon = t_closeness(train_data_anon, [], quasi_ident, sens_att, cfg.fixed_k, anon_parameter, supp_level, hierarchies)

                            # Assert that the level of t-closeness is satisfied
                            actual_t_closeness = pycanon.anonymity.t_closeness(train_data_anon, quasi_ident, [sens_att], True)
                            assert actual_t_closeness <= anon_parameter, f"t-closeness constraint not met: Expected <= {anon_parameter}, but got {actual_t_closeness:.2f}"

                        # Get generalization levels of the training set to apply the same to the test set
                        generalization_levels = get_generalization_levels(train_data_anon, quasi_ident, hierarchies)

                        # Apply the same generalization levels to the test data (Except for the protected attribute: for fairness measurements)
                        for col in set(quasi_ident) - {protected_att}:
                            level = generalization_levels.get(col)
                            
                            if level is not None:
                                # Retrieve the mapping dictionary for this level
                                hierarchy_mapping = dict(zip(hierarchies[col][0], hierarchies[col][level]))
                                
                                # Apply the mapping to the test data
                                test_data[col] = test_data[col].map(hierarchy_mapping)

                        # Separate features and target
                        X_train, y_train, X_test, y_test = get_train_test_data(train_data_anon, test_data, sens_att)

                        # Train the model
                        model = XGB(random_state=current_seed, n_jobs=-1)
                        model.fit(X_train, y_train)

                        # Get fairness/utility metrics
                        df_fm = test_data.copy()
                        df_fm['y_pred'] = np.round(model.predict(X_test)).reshape(-1).astype(int)
                        dic_metrics = get_metrics(df_fm, protected_att, sens_att)

                        # Compute individual fairness metrics
                        soft_ypred = model.predict_proba(X_test)
                        
                        asf_score = similarity_fairness(soft_ypred, X_test.values)
                        dic_metrics['ASF'] = np.abs(asf_score)

                        alf_score = lipschitz_fairness(soft_ypred, X_test.values)
                        dic_metrics['ALF'] = np.abs(alf_score)

                        ncf_score = neighborhood_consistency_fairness(soft_ypred, X_test.values)
                        dic_metrics['NCF'] = np.abs(ncf_score)
                        print(dic_metrics)

                        # Write results to csv
                        write_target_distribution_results_to_csv([current_seed, protected_att, sens_att, method, anon_parameter, threshold_target] + list(dic_metrics.values()), dataset=dataset)

                        SEED += 1

                    except Exception as e:
                            print(f"An error occurred for SEED {SEED}, current_seed {current_seed} k {cfg.fixed_k}: {e}")
                            continue
                    
                    finally:
                        # In all cases, increment current_seed for a new attempt
                        current_seed += 1
                
                print('-------------------------------------------------------------\n')
            print('=============================================================\n')
        print('#############################################################\n')
    print('=============================================================\n')
ray.shutdown()

2025-01-15 17:17:50,801	INFO worker.py:1749 -- Started a local Ray instance.


decile for threshold target: 10
Method: k-anonymity, Parameter: 10
Dataset: adult with Protected Attribute: gender
SEED: 0, current_seed: 0
{'SPD': 0.07141416840934867, 'EOD': 0.036281225867900724, 'MAD': 0.06467391470750716, 'PED': 0.11938095238095237, 'PRD': 0.040176366843033495, 'ACC': 0.9358778625954198, 'f1': 0.9657741559953434, 'Precision': 0.9512670565302144, 'Recall': 0.980730582811207, 'ROC_AUC': 0.6914904672143236, 'CM': array([[ 286,  425],
       [ 163, 8296]], dtype=int64), 'ASF': 0.061176445, 'ALF': 2.752630232384198, 'NCF': 0.043962725}
SEED: 1, current_seed: 1
{'SPD': 0.05887115429100165, 'EOD': 0.024153052141925757, 'MAD': 0.061965070552856716, 'PED': 0.05281385281385287, 'PRD': 0.048496168775013504, 'ACC': 0.9363140676117775, 'f1': 0.9659514925373134, 'Precision': 0.9525123605841095, 'Recall': 0.9797752808988764, 'ROC_AUC': 0.7010764516382494, 'CM': array([[ 302,  413],
       [ 171, 8284]], dtype=int64), 'ASF': 0.064247504, 'ALF': 3.6389759624152402, 'NCF': 0.0446203

c:\Users\heber\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'SPD': -0.07032590051457976, 'EOD': -0.34782608695652173, 'MAD': -0.16128928760936356, 'PED': -0.04130352406214475, 'PRD': -0.4682926829268293, 'ACC': 0.7908396946564885, 'f1': 0.0909952606635071, 'Precision': 0.4682926829268293, 'Recall': 0.050393700787401574, 'ROC_AUC': 0.5176951298155865, 'CM': array([[7156,  109],
       [1809,   96]], dtype=int64), 'ASF': 6.152781e-05, 'ALF': 0.451119065284729, 'NCF': 6.1527804e-05}
SEED: 8, current_seed: 8
The data verifies k-anonymity with k=10
{'SPD': 0.21738920827759098, 'EOD': 0.25711280433053696, 'MAD': -0.14087753552446036, 'PED': 0.13102506153029952, 'PRD': -0.023500239961606106, 'ACC': 0.822246455834242, 'f1': 0.5627682403433476, 'Precision': 0.5837506956037841, 'Recall': 0.5432418436043501, 'ROC_AUC': 0.7199563272449157, 'CM': array([[6491,  748],
       [ 882, 1049]], dtype=int64), 'ASF': 0.004240532, 'ALF': 1.5791306402609484, 'NCF': 0.004231326}
SEED: 9, current_seed: 9
The data verifies k-anonymity with k=10
{'SPD': 0.21782837866601

c:\Users\heber\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

{'SPD': 0.0008866371120962635, 'EOD': 0.00228310502283105, 'MAD': -0.04937550447664807, 'PED': 0.0007123521869212138, 'PRD': 0.2857142857142857, 'ACC': 0.8955288985823336, 'f1': 0.004158004158004158, 'Precision': 0.2857142857142857, 'Recall': 0.0020942408376963353, 'ROC_AUC': 0.5007427990554884, 'CM': array([[8210,    5],
       [ 953,    2]], dtype=int64), 'ASF': 0.018438363, 'ALF': 1.4962064027786255, 'NCF': 0.017535483}
SEED: 38, current_seed: 38
The data verifies k-anonymity with k=10


c:\Users\heber\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'SPD': -0.03250188964474679, 'EOD': -0.17582417582417584, 'MAD': -0.030714396969016566, 'PED': -0.021915584415584416, 'PRD': -0.37209302325581395, 'ACC': 0.896619411123228, 'f1': 0.0326530612244898, 'Precision': 0.37209302325581395, 'Recall': 0.017075773745997867, 'ROC_AUC': 0.5068981443733026, 'CM': array([[8206,   27],
       [ 921,   16]], dtype=int64), 'ASF': 0.0054050465, 'ALF': 2.0415918827056885, 'NCF': 0.0053911814}
SEED: 39, current_seed: 39
The data verifies k-anonymity with k=10


c:\Users\heber\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\heber\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\heber\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'SPD': 0.0, 'EOD': 0.0, 'MAD': -0.043784848484848427, 'PED': 0.0, 'PRD': 0.0, 'ACC': 0.8949836423118865, 'f1': 0.0, 'Precision': 0.0, 'Recall': 0.0, 'ROC_AUC': 0.5, 'CM': array([[8207,    0],
       [ 963,    0]], dtype=int64), 'ASF': 0.0042419652, 'ALF': 1.4243874549865723, 'NCF': 0.004224176}
-------------------------------------------------------------


#############################################################




0.0

## Run baseline (k=1)

In [2]:
# warning filters
import warnings
warnings.filterwarnings("ignore", message="Pandas requires version")
warnings.filterwarnings("ignore", message="A NumPy version >=")

# general imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# anonymity library
import pycanon
from anjana.anonymity import k_anonymity, l_diversity, t_closeness

# ML models
from xgboost import XGBClassifier as XGB

# our generic functions
from utils import get_metrics, write_target_distribution_results_to_csv, get_generalization_levels, get_train_test_data

# our data-specific functions
from utils import clean_process_data, get_hierarchies

# our individual fairness metrics
from utils import lipschitz_fairness, similarity_fairness, neighborhood_consistency_fairness

# our config file
import config_experiments as cfg

# import folkatables
import folktables
from folktables import ACSDataSource

# ray for parallel processing (individual fairness is computationally expensive)
import ray 
import os
ray.init(num_cpus=os.cpu_count(), ignore_reinit_error=True)

# Get parameters from config file
lst_dataset = cfg.lst_dataset
lst_sensitive_attributes = cfg.lst_sensitive_attributes
max_seed = cfg.max_seed
test_size = cfg.test_size
fraction = cfg.fixed_fraction
supp_level = cfg.supp_level[1]
lst_deciles = cfg.lst_deciles
dic_methods_parameters = {
                         'k-anonymity': 1,
                         } 

# Loop over several decile values
for decile in lst_deciles:
    print(f"decile for threshold target: {decile}")

    for dataset in ['adult']:#lst_dataset:

        # Main execution
        write_target_distribution_results_to_csv([], dataset=dataset, header=True)
        
        for protected_att in lst_sensitive_attributes[dataset]:

            # Loop over the three anonymization methods
            for method, anon_parameter in dic_methods_parameters.items():
                print(f"Method: {method}, Parameter: {anon_parameter}") 

                # read data
                if dataset == 'adult':

                    # Sensitive/target and protected attributes
                    sens_att = "income"
                    
                    # Read and process the data
                    data = pd.read_csv("adult_reconstruction.csv")
                    threshold_target = int(np.percentile(data[sens_att], decile))
                    data = clean_process_data(data, dataset, sens_att, protected_att, threshold_target)

                elif dataset == 'ACSIncome':

                    # Sensitive/target
                    sens_att = 'PINCP'

                    # Read and process the data
                    data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
                    acs_data = data_source.get_data()
                    Our_ACSIncome = folktables.BasicProblem(
                                                            features=[
                                                                    'AGEP',
                                                                    'COW',
                                                                    'SCHL',
                                                                    'MAR',
                                                                    'OCCP',
                                                                    'POBP',
                                                                    'RELP',
                                                                    'WKHP',
                                                                    'SEX',
                                                                    'RAC1P',
                                                                    ],
                                                            target='PINCP',
                                                            target_transform=lambda x: x,    
                                                            group=protected_att,
                                                            preprocess=folktables.adult_filter,
                                                            postprocess=lambda x: np.nan_to_num(x, -1),
                                                            )
                    features, target, _ = Our_ACSIncome.df_to_pandas(acs_data)
                    data = pd.concat([features, target.astype(int)], axis=1)
                    threshold_target = int(np.percentile(data[sens_att], decile))
                    full_data = clean_process_data(data, dataset, sens_att, protected_att, threshold_target)

                print("Dataset: {} with Protected Attribute: {}".format(dataset, protected_att))

                # Loop over several seeds            
                SEED = 0
                current_seed = 0
                while SEED < max_seed:
                    print(f"SEED: {SEED}, current_seed: {current_seed}")
                    data = full_data.sample(frac=fraction, random_state=current_seed) if dataset == 'ACSIncome' else data 

                    try:
                        # Split into train and test data
                        train_data, test_data = train_test_split(data, test_size=test_size, random_state=current_seed)

                        # Separate features and target
                        X_train, y_train, X_test, y_test = get_train_test_data(train_data, test_data, sens_att)

                        # Train the model
                        model = XGB(random_state=current_seed, n_jobs=-1)
                        model.fit(X_train, y_train)

                        # Get fairness/utility metrics
                        df_fm = test_data.copy()
                        df_fm['y_pred'] = np.round(model.predict(X_test)).reshape(-1).astype(int)
                        dic_metrics = get_metrics(df_fm, protected_att, sens_att)

                        # Compute individual fairness metrics
                        soft_ypred = model.predict_proba(X_test)
                        
                        asf_score = similarity_fairness(soft_ypred, X_test.values)
                        dic_metrics['ASF'] = np.abs(asf_score)

                        alf_score = lipschitz_fairness(soft_ypred, X_test.values)
                        dic_metrics['ALF'] = np.abs(alf_score)

                        ncf_score = neighborhood_consistency_fairness(soft_ypred, X_test.values)
                        dic_metrics['NCF'] = np.abs(ncf_score)
                        print(dic_metrics)

                        # Write results to csv
                        write_target_distribution_results_to_csv([current_seed, protected_att, sens_att, method, anon_parameter, threshold_target] + list(dic_metrics.values()), dataset=dataset)

                        SEED += 1

                    except Exception as e:
                            print(f"An error occurred for SEED {SEED}, current_seed {current_seed} k {cfg.fixed_k}: {e}")
                            continue
                    
                    finally:
                        # In all cases, increment current_seed for a new attempt
                        current_seed += 1
                
                print('-------------------------------------------------------------\n')
            print('=============================================================\n')
        print('#############################################################\n')
    print('=============================================================\n')
ray.shutdown()

2025-01-16 03:30:42,938	INFO worker.py:1749 -- Started a local Ray instance.


decile for threshold target: 10
Method: k-anonymity, Parameter: 1
Dataset: adult with Protected Attribute: gender
SEED: 0, current_seed: 0
{'SPD': 0.06044205146733017, 'EOD': 0.02488759502588689, 'MAD': 0.05582341561008597, 'PED': 0.08869047619047621, 'PRD': 0.040356597836076036, 'ACC': 0.9411123227917121, 'f1': 0.9685973482205164, 'Precision': 0.9531875929953073, 'Recall': 0.9845135358789455, 'ROC_AUC': 0.7046337018353939, 'CM': array([[ 302,  409],
       [ 131, 8328]], dtype=int64), 'ASF': 0.47500828, 'ALF': 1.457970805172348, 'NCF': 0.056687873}
SEED: 1, current_seed: 1
{'SPD': 0.05157009021512837, 'EOD': 0.01881720972182399, 'MAD': 0.060128961369419365, 'PED': 0.004761904761904745, 'PRD': 0.050793329552280664, 'ACC': 0.9399127589967284, 'f1': 0.9678848283499446, 'Precision': 0.9541484716157205, 'Recall': 0.9820224719101124, 'ROC_AUC': 0.7119902569340772, 'CM': array([[ 316,  399],
       [ 152, 8303]], dtype=int64), 'ASF': 0.48064798, 'ALF': 1.8203837081512917, 'NCF': 0.057571165}